In [1]:
import pandas as pd
import numpy as np

In [2]:
def seasonLabel(row):
    if row['month'] in [3,4,5]:
        return 'spring'
    if row['month'] in [6,7,8]:
        return 'summer'
    if row['month'] in [9,10,11]:
        return 'fall'
    if row['month'] in [12,1,2]:
        return 'winter'

In [3]:
from datetime import datetime as dt

In [4]:
# oct 6 to nov 9
data = pd.read_csv("Police Incidents 2019 (1).csv", usecols=['Precinct', "reportedDateTime",'Description','Neighborhood'])
data['reportedDateTime'] = pd.to_datetime(data['reportedDateTime'])
data = data[data['reportedDateTime'] > '2019-01-01']
data['reportedDateTime'] = data['reportedDateTime'].dt.date
data = data.rename(columns={"reportedDateTime": "ReportedDate"})

data = data.groupby(['ReportedDate', 'Precinct', 'Description']).count().reset_index()
data = data.rename(columns={'Neighborhood': 'count_incidents'})
data.head()

,ReportedDate,Precinct,Description,count_incidents
0,2019-01-01,1,AUTOMOBILE THEFT,1
1,2019-01-01,1,BURGLARY OF DWELLING,1
2,2019-01-01,1,OTHER THEFT,3
3,2019-01-01,1,ROBBERY PER AGG,1
4,2019-01-01,1,SHOPLIFTING,1


In [5]:
# nov 7 to dec 5
data2 = pd.read_csv("new.csv")
data2 = data2.drop(columns=['Unnamed: 0'])
data2 = data2.rename(columns = {'reportedDate':'ReportedDate', 'precinct':'Precinct', 'count_of_crimes':'count_incidents', 'description':'Description'})
data2.head()

,ReportedDate,Precinct,Description,count_incidents
0,2019-11-07,1,Steal From Person,8
1,2019-11-07,2,Steal From Person,6
2,2019-11-07,2,Steal From Store,1
3,2019-11-07,3,Assault/Sexual,4
4,2019-11-07,3,Steal From Person,15


In [6]:
data = pd.concat([data, data2])
data.reset_index(inplace=True)


In [8]:
data.to_csv('rik.csv')

In [9]:
data.head()

,index,ReportedDate,Precinct,Description,count_incidents
0,0,2019-01-01,1,AUTOMOBILE THEFT,1
1,1,2019-01-01,1,BURGLARY OF DWELLING,1
2,2,2019-01-01,1,OTHER THEFT,3
3,3,2019-01-01,1,ROBBERY PER AGG,1
4,4,2019-01-01,1,SHOPLIFTING,1


In [ ]:
# Dates
data["ReportedDate"] = pd.to_datetime(data["ReportedDate"])
data["year"] = data["ReportedDate"].dt.year
data["month"] = data["ReportedDate"].dt.month
data["day"] = data["ReportedDate"].dt.day
data["week_of_year"] = data["ReportedDate"].dt.week
data["dayofweek"] = data["ReportedDate"].dt.dayofweek # 0 (sunday) - 6
data["season"] = data.apply(lambda row:seasonLabel(row), axis=1)  
data["ReportedDate"] = data["ReportedDate"].dt.date
data = data.drop(columns='index')
data.head(10)

In [10]:
weather = pd.read_csv('weather_2_2019_precip_minneapolis.csv', usecols=['DATE', 'PRCP','SNOW', 'TAVG', 'SNWD'])
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather['DATE'] = weather['DATE'].dt.date
weather['Full_Moon'] = np.where((weather['DATE'] == '2019-10-13') | (weather['DATE'] == '2019-10-13'), 1, 0)
weather.head()
weather['DATE'].max()

datetime.date(2019, 11, 15)

In [11]:
full_train = pd.merge(data, weather, left_on='ReportedDate', right_on='DATE', how='left')

poverty = pd.read_csv('poverty.csv', usecols=['precinct', 'Poverty_Count', 'Poverty_Under_18'])
full_train = pd.merge(full_train, poverty, left_on='Precinct', right_on='precinct', how='left')

full_train.head()

,ReportedDate,Precinct,Description,count_incidents,year,month,day,week_of_year,dayofweek,season,DATE,PRCP,SNOW,SNWD,TAVG,Full_Moon,precinct,Poverty_Count,Poverty_Under_18
0,2019-10-06,1,AUTOMOBILE THEFT,3,2019,10,6,40,6,fall,2019-10-06,0.0,0.0,0.0,54.0,0.0,1,1.783853e+06,62896.115282
1,2019-10-06,1,CSC - RAPE,1,2019,10,6,40,6,fall,2019-10-06,0.0,0.0,0.0,54.0,0.0,1,1.783853e+06,62896.115282
2,2019-10-06,1,OTHER THEFT,3,2019,10,6,40,6,fall,2019-10-06,0.0,0.0,0.0,54.0,0.0,1,1.783853e+06,62896.115282
3,2019-10-06,1,THEFT FROM MOTR VEHC,1,2019,10,6,40,6,fall,2019-10-06,0.0,0.0,0.0,54.0,0.0,1,1.783853e+06,62896.115282
4,2019-10-06,1,THEFT FROM PERSON SNATCH/GRAB,1,2019,10,6,40,6,fall,2019-10-06,0.0,0.0,0.0,54.0,0.0,1,1.783853e+06,62896.115282


In [12]:
full_train.to_csv('full_train.csv')

In [19]:
data_2017 = pd.read_csv("Police Incidents 2017.csv", usecols=['Precinct', "ReportedDate",'Description','Neighborhood'])

data_2017['Description'] = np.where(data_2017['Description'].str.contains(('theft|robbery|BURGLARY|SHOPLIFTING|POCKET|LOOTING'), case=False, na=False), 'Theft',data_2017['Description'])
data_2017['Description'] = np.where(data_2017['Description'].str.contains(('murder'), case=False, na=False), 'Murder',data_2017['Description'])
data_2017['Description'] = np.where(data_2017['Description'].str.contains(('DOMES|DOMESTIC|SODOMY|PENETRATE|RAPE'), case=False, na=False), 'Domestic/Sexual',data_2017['Description'])
data_2017['Description'] = np.where(data_2017['Description'].str.contains(('asslt|aslt'), case=False, na=False), 'Assault',data_2017['Description'])
data_2017['Description'] = np.where(data_2017['Description'].str.contains(('obs|arson'), case=False, na=False), 'Other',data_2017['Description'])

# Dates
data_2017["ReportedDate"] = pd.to_datetime(data_2017["ReportedDate"])
data_2017["year"] = data_2017["ReportedDate"].dt.year
data_2017["month"] = data_2017["ReportedDate"].dt.month
data_2017["day"] = data_2017["ReportedDate"].dt.day

data_2017["week_of_year"] = data_2017["ReportedDate"].dt.week
data_2017["dayofweek"] = data_2017["ReportedDate"].dt.dayofweek # 0 (sunday) - 6
data_2017["season"] = data_2017.apply(lambda row:seasonLabel(row), axis=1)  

# Grouping
data_2017 = data_2017.groupby(['year','month','day','season', 'week_of_year','dayofweek', 'Precinct', 'Neighborhood', 'Description']).agg("count")
data_2017 = data_2017.reset_index().rename(columns={"ReportedDate": "count_of_crimes"})
data_2017

,year,month,day,season,week_of_year,dayofweek,Precinct,Neighborhood,Description,count_of_crimes
0,2017,1,30,winter,5,0,1.0,CEDAR RIVERSIDE,Theft,1
1,2017,2,28,winter,9,1,1.0,DOWNTOWN WEST,Domestic/Sexual,1
2,2017,3,15,spring,11,2,3.0,VENTURA VILLAGE,Domestic/Sexual,1
3,2017,3,20,spring,12,0,4.0,HAWTHORNE,Theft,1
4,2017,3,20,spring,12,0,4.0,LIND - BOHANON,Theft,1
...,...,...,...,...,...,...,...,...,...,...
727,2017,12,26,winter,52,1,4.0,HAWTHORNE,Theft,1
728,2017,12,26,winter,52,1,4.0,LIND - BOHANON,Domestic/Sexual,1
729,2017,12,26,winter,52,1,4.0,NEAR - NORTH,Theft,1
730,2017,12,26,winter,52,1,4.0,SUMNER - GLENWOOD,Domestic/Sexual,1


In [20]:
data_2018_pims = pd.read_csv("Police Incidents 2018 PIMS.csv", usecols=['Precinct', "reportedDateTime",'Description','Neighborhood'])
data_2018_pims = data_2018_pims.rename(columns={"reportedDateTime": "ReportedDate"})

data_2018_pims['Description'] = np.where(data_2018_pims['Description'].str.contains(('theft|robbery|BURGLARY|SHOPLIFTING|POCKET|LOOTING'), case=False, na=False), 'Theft',data_2018_pims['Description'])
data_2018_pims['Description'] = np.where(data_2018_pims['Description'].str.contains(('murder'), case=False, na=False), 'Murder',data_2018_pims['Description'])
data_2018_pims['Description'] = np.where(data_2018_pims['Description'].str.contains(('DOMES|DOMESTIC|SODOMY|PENETRATE|RAPE'), case=False, na=False), 'Domestic/Sexual',data_2018_pims['Description'])
data_2018_pims['Description'] = np.where(data_2018_pims['Description'].str.contains(('asslt|aslt'), case=False, na=False), 'Assault',data_2018_pims['Description'])
data_2018_pims['Description'] = np.where(data_2018_pims['Description'].str.contains(('obs|arson|computer|gas'), case=False, na=False), 'Other',data_2018_pims['Description'])

# Dates
data_2018_pims["ReportedDate"] = pd.to_datetime(data_2018_pims["ReportedDate"])
data_2018_pims["year"] = data_2018_pims["ReportedDate"].dt.year
data_2018_pims["month"] = data_2018_pims["ReportedDate"].dt.month
data_2018_pims["day"] = data_2018_pims["ReportedDate"].dt.day

data_2018_pims["week_of_year"] = data_2018_pims["ReportedDate"].dt.week
data_2018_pims["dayofweek"] = data_2018_pims["ReportedDate"].dt.dayofweek # 0 (sunday) - 6
data_2018_pims["season"] = data_2018_pims.apply(lambda row:seasonLabel(row), axis=1)  

# Grouping
data_2018_pims = data_2018_pims.groupby(['year','month','day','season', 'week_of_year','dayofweek', 'Precinct', 'Neighborhood', 'Description']).agg("count")
data_2018_pims = data_2018_pims.reset_index().rename(columns={"ReportedDate": "count_of_crimes"})
data_2018_pims

,year,month,day,season,week_of_year,dayofweek,Precinct,Neighborhood,Description,count_of_crimes
0,2018,6,4,summer,23,0,1,Loring Park,Domestic/Sexual,1
1,2018,6,5,summer,23,1,1,Elliot Park,Theft,1
2,2018,6,5,summer,23,1,3,Howe,Theft,1
3,2018,6,5,summer,23,1,4,Hawthorne,Theft,1
4,2018,6,5,summer,23,1,5,Lowry Hill East,Theft,1
...,...,...,...,...,...,...,...,...,...,...
7642,2018,12,31,winter,1,0,5,Fulton,Theft,1
7643,2018,12,31,winter,1,0,5,Lowry Hill East,Theft,1
7644,2018,12,31,winter,1,0,5,Lynnhurst,Theft,1
7645,2018,12,31,winter,1,0,5,Whittier,Theft,1


In [51]:
data_2018 = pd.read_csv("Police Incidents 2018.csv", usecols=['Precinct', "ReportedDate",'Description','Neighborhood'])

data_2018['Description'] = np.where(data_2018['Description'].str.contains(('theft|robbery|BURGLARY|SHOPLIFTING|POCKET|LOOTING'), case=False, na=False), 'Theft',data_2018['Description'])
data_2018['Description'] = np.where(data_2018['Description'].str.contains(('murder'), case=False, na=False), 'Murder',data_2018['Description'])
data_2018['Description'] = np.where(data_2018['Description'].str.contains(('DOMES|DOMESTIC|SODOMY|PENETRATE|RAPE|disarm'), case=False, na=False), 'Domestic/Sexual',data_2018['Description'])
data_2018['Description'] = np.where(data_2018['Description'].str.contains(('asslt|aslt'), case=False, na=False), 'Assault',data_2018['Description'])
data_2018['Description'] = np.where(data_2018['Description'].str.contains(('obs|arson|computer|gas'), case=False, na=False), 'Other',data_2018['Description'])

# Dates
data_2018["ReportedDate"] = pd.to_datetime(data_2018["ReportedDate"])
data_2018["year"] = data_2018["ReportedDate"].dt.year
data_2018["month"] = data_2018["ReportedDate"].dt.month
data_2018["day"] = data_2018["ReportedDate"].dt.day

data_2018["week_of_year"] = data_2018["ReportedDate"].dt.week
data_2018["dayofweek"] = data_2018["ReportedDate"].dt.dayofweek # 0 (sunday) - 6
data_2018["season"] = data_2018.apply(lambda row:seasonLabel(row), axis=1)  

# Grouping
data_2018 = data_2018.groupby(['year','month','day','season', 'week_of_year','dayofweek', 'Precinct', 'Neighborhood', 'Description']).agg("count")
data_2018 = data_2018.reset_index().rename(columns={"ReportedDate": "count_of_crimes"})

In [22]:
#pd.concat([data_2017, data_2018])
all_data = pd.concat([data_2017, data_2018, data_2018_pims, data2019])
all_data['Neighborhood'] = all_data['Neighborhood'].str.lower().str.capitalize()

In [123]:
all_data.to_csv('all_data.csv')

In [3]:
new_data = pd.read_csv('mps.csv', usecols=['Total House Units','Average household size - count', 'Neighborhood', 'Median household income (2017 dollars, averaged) - count', 'Poverty - All people for whom poverty status is determined - count', 'Poverty - 17 years and younger (percent of people under age 18) - count'])
new_data = new_data.rename(columns={"Total House Units": "Houses", 'Median household income (2017 dollars, averaged) - count': 'Median_income'})
new_data = new_data.rename(columns={"Average household size - count": "House_Size", "Poverty - All people for whom poverty status is determined - count": "Poverty_Count"})
new_data = new_data.rename(columns={"Poverty - 17 years and younger (percent of people under age 18) - count": "Poverty_Under_18"})
new_data = new_data[["Neighborhood", "Poverty_Count", "Poverty_Under_18"]]
new_data['Neighborhood'] = new_data['Neighborhood'].str.lower()

In [5]:
data

,Neighborhood,Poverty_Count,Poverty_Under_18
0,Corcoran,4150.985676,277.795818
1,Kenny,3599.899592,63.333395
2,Cooper,3684.677990,26.953754
3,East harriet,3552.196744,31.816250
4,Cedar-isles-dean,2848.217590,9.956482
...,...,...,...
63,Battle creek-highwood,22775.913910,1757.883954
64,Macalester-groveland,16647.283610,92.415376
65,Payne-phalen,31823.697670,3318.841438
66,Greater east side,29750.695950,3163.635794


In [49]:
#18k
with_new = pd.merge(data, all_data, on='Neighborhood', how='right')

In [63]:
all_data = with_new.round({'Poverty_Count': 0, 'Poverty_Under_18': 0})
all_data.sort_values(by=['year','month','day','Neighborhood'])

,Neighborhood,Poverty_Count,Poverty_Under_18,year,month,day,season,week_of_year,dayofweek,Precinct,Description,count_of_crimes
10951,Cedar riverside,7525.0,1022.0,2017,1,30,winter,5,0,1,Theft,1
13240,Downtown west,6418.0,64.0,2017,2,28,winter,9,1,1,Domestic/Sexual,1
7738,Ventura village,6406.0,1035.0,2017,3,15,spring,11,2,3,Domestic/Sexual,1
17599,Hawthorne,NaN,NaN,2017,3,20,spring,12,0,4,Theft,1
851,Lind-bohanon,5202.0,434.0,2017,3,20,spring,12,0,4,Theft,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3150,Phillips west,5512.0,1111.0,2019,11,9,fall,45,5,3,Domestic/Sexual,1
21497,Regina,NaN,NaN,2019,11,9,fall,45,5,3,Theft,1
8298,Ventura village,6406.0,1035.0,2019,11,9,fall,45,5,3,Theft,2
3992,Webber-camden,5290.0,667.0,2019,11,9,fall,45,5,4,Theft,2
